In [1]:
# the warnings are total BS e.g: The name reduce_all is deprecated. Please use reduce_all instead 
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [2]:
import time

import tensorflow as tf
import numpy as np
import ray

from sklearn.model_selection import train_test_split

from model.ram import RecurrentAttentionModel

from data.augmented_mnist import minibatcher
from data.augmented_mnist import get_mnist

W0417 12:53:52.097383 140588183955200 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/math/root_search.py:51: The name reduce_all is deprecated. Please use reduce_all instead.

W0417 12:53:52.099782 140588183955200 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/bijectors/masked_autoregressive.py:412: The name nn.relu is deprecated. Please use nn.relu instead.

W0417 12:53:52.100424 140588183955200 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/bijectors/masked_autoregressive.py:529: The name keras.layers.Layer is deprecated. Please use keras.layers.Layer instead.

W0417 12:53:52.153022 140588183955200 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/edward2/random_variable.py:191: The name Tensor is deprecated. Please use Tensor instead.

W0417 12:53:52.154199 140588183955200 

In [3]:
(X_train, y_train),(X_test, y_test) = get_mnist(True, True, False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print(X_train.shape, y_train.shape, np.max(X_train), np.min(X_train))
print(X_val.shape, y_val.shape, np.max(X_val), np.min(X_val))
print(X_test.shape, y_test.shape, np.max(X_test), np.min(X_test))

W0417 12:53:52.415025 140588183955200 deprecation_wrapper.py:76] From /tf/workspace/recurrent-visual-attention-model/data/augmented_mnist.py:64: The name keras.datasets.mnist.load_data is deprecated. Please use keras.datasets.mnist.load_data instead.

W0417 12:53:52.838382 140588183955200 deprecation_wrapper.py:76] From /tf/workspace/recurrent-visual-attention-model/data/augmented_mnist.py:78: The name keras.utils.to_categorical is deprecated. Please use keras.utils.to_categorical instead.



(45000, 28, 28, 1) (45000, 10) 1.0 0.0
(15000, 28, 28, 1) (15000, 10) 1.0 0.0
(10000, 28, 28, 1) (10000, 10) 1.0 0.0


In [4]:
@ray.remote
def train(hyperparameter, X_train, y_train, X_val, y_val):
    ram = RecurrentAttentionModel(time_steps=7,
                                  n_glimpses=1, 
                                  glimpse_size=8,
                                  num_classes=10,
                                  max_gradient_norm=5.0,
                                  std=hyperparameter["std"])
    optimizer = tf.keras.optimizers.Adam(hyperparameter["learning_rate"])
    for e in range(10):
        # trainings step
        batcher = minibatcher(X_train, y_train, 500, True)
        for X, y in batcher:
            with tf.GradientTape() as tape:
                logits = ram(X)
                hybrid_loss, _, _, _ = ram.hybrid_loss(logits, y)

                gradients = tape.gradient(hybrid_loss, ram.trainable_variables)
                optimizer.apply_gradients(zip(gradients, ram.trainable_variables))

        # testing step
        batcher = minibatcher(X_val, y_val, 500, True)
        accuracys = []
        for X, y in batcher:
            logits = ram(X)
            accuracy, _, _ = ram.predict(logits, y)
            accuracy = accuracy.numpy()
            accuracys.append(accuracy)
    return np.mean(accuracys)

In [5]:
def generate_hyperparameters():
    # Randomly choose values for the hyperparameters.
    return {"learning_rate": 10 ** -np.random.uniform(0, 8),
            "std": 10 ** -np.random.uniform(0, 2)}

In [6]:
ray.init()

# Launch a experiments.
remaining_ids = []
hyperparameters = generate_hyperparameters()
remaining_ids.append(train.remote(hyperparameters, X_train, y_train, X_val, y_val))

# Whenever a new experiment finishes, print the value and start a new
# experiment.
for i in range(10):
    ready_ids, remaining_ids = ray.wait(remaining_ids, num_returns=1)
    accuracy = ray.get(ready_ids[0])
    print("Accuracy is {}".format(accuracy), "Hyperparameter:", hyperparameters)
    # Start a new experiment.
    hyperparameters = generate_hyperparameters()
    remaining_ids.append(train.remote(hyperparameters, X_train, y_train, X_val, y_val))

2019-04-17 12:53:52,971	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-17_12-53-52_9743/logs.
2019-04-17 12:53:53,098	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:61304 to respond...
2019-04-17 12:53:53,226	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:16930 to respond...
2019-04-17 12:53:53,229	INFO services.py:760 -- Starting Redis shard with 10.0 GB max memory.
2019-04-17 12:53:53,269	WARNING services.py:1236 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-04-17 12:53:53,270	WARNING services.py:1261 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you 

(pid=9805) WARNING: Logging before flag parsing goes to stderr.
(pid=9805) W0417 12:53:56.630001 140654403319552 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/math/root_search.py:51: The name reduce_all is deprecated. Please use reduce_all instead.
(pid=9805) 
(pid=9805) W0417 12:53:56.630887 140654403319552 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/bijectors/masked_autoregressive.py:412: The name nn.relu is deprecated. Please use nn.relu instead.
(pid=9805) 
(pid=9805) W0417 12:53:56.631088 140654403319552 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/bijectors/masked_autoregressive.py:529: The name keras.layers.Layer is deprecated. Please use keras.layers.Layer instead.
(pid=9805) 
(pid=9795) WARNING: Logging before flag parsing goes to stderr.
(pid=9795) W0417 12:53:56.649762 140588611417856 deprecation_wrapper.py:76

(pid=9806) W0417 12:53:56.879309 139694948947712 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/edward2/random_variable.py:191: The name Tensor is deprecated. Please use Tensor instead.
(pid=9806) 
(pid=9806) W0417 12:53:56.880479 139694948947712 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/edward2/random_variable.py:325: The name register_tensor_conversion_function is deprecated. Please use register_tensor_conversion_function instead.
(pid=9806) 
(pid=9805) W0417 12:53:56.840920 140654403319552 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/layers/distribution_layer.py:89: The name keras.layers.Lambda is deprecated. Please use keras.layers.Lambda instead.
(pid=9805) 
(pid=9805) W0417 12:53:56.841606 140654403319552 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/layers/di

(pid=9802) W0417 12:53:57.074195 140130090014464 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/optimizer/sgld.py:34: The name keras.optimizers.Optimizer is deprecated. Please use keras.optimizers.Optimizer instead.
(pid=9802) 
(pid=9802) W0417 12:53:57.079264 140130090014464 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/positive_semidefinite_kernels/internal/util.py:89: The name custom_gradient is deprecated. Please use custom_gradient instead.
(pid=9802) 
(pid=9800) W0417 12:53:57.073637 140483705661184 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/edward2/random_variable.py:191: The name Tensor is deprecated. Please use Tensor instead.
(pid=9800) 
(pid=9800) W0417 12:53:57.074104 140483705661184 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/edward2/random_variable.py:

(pid=9807) W0417 12:53:57.267657 140617017554688 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/layers/dense_variational_v2.py:151: The name convert_to_tensor is deprecated. Please use convert_to_tensor instead.
(pid=9807) 
(pid=9807) W0417 12:53:57.269789 140617017554688 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/layers/distribution_layer.py:89: The name keras.layers.Lambda is deprecated. Please use keras.layers.Lambda instead.
(pid=9807) 
(pid=9807) W0417 12:53:57.270059 140617017554688 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probability/python/layers/distribution_layer.py:1136: The name keras.regularizers.Regularizer is deprecated. Please use keras.regularizers.Regularizer instead.
(pid=9807) 
(pid=9807) W0417 12:53:57.270357 140617017554688 deprecation_wrapper.py:76] From /usr/local/lib/python3.5/dist-packages/tensorflow_probabili

(pid=10134) W0417 12:53:57.475478 140415362680576 deprecation_wrapper.py:76] From /tf/workspace/recurrent-visual-attention-model/model/ram.py:4: The name keras.Model is deprecated. Please use keras.models.Model instead.
(pid=10134) 
(pid=10134) W0417 12:53:57.476043 140415362680576 deprecation_wrapper.py:76] From /tf/workspace/recurrent-visual-attention-model/model/ram.py:5: The name keras.layers.LSTMCell is deprecated. Please use keras.layers.LSTMCell instead.
(pid=10134) 
(pid=9800) 2019-04-17 12:53:57.648586: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1485] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
(pid=9800) W0417 12:53:57.873097 140483

Exception in thread ray_import_thread:
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/ray/import_thread.py", line 70, in _run
    msg = import_pubsub_client.get_message()
  File "/usr/local/lib/python3.5/dist-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/usr/local/lib/python3.5/dist-packages/redis/client.py", line 3036, in parse_response
    return self._execute(connection, connection.read_r

KeyboardInterrupt: 